In [38]:
import re
import math
import copy
from collections import deque

# Utils

In [39]:
def find_qubit(qubit, grid):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == qubit:
                return (i, j)
    return None

def ManDist(neighbor, qubit):
    return abs(neighbor[0] - qubit[0]) + abs(neighbor[1] - qubit[1]) - 1

def DoA(new_qubit, qubit_pos, grid, adj_mat):
    qubit = grid[qubit_pos[0]][qubit_pos[1]]
    return adj_mat[new_qubit-1][qubit-1] + adj_mat[qubit-1][new_qubit-1]

def update_neighbor_list(grid, X, Y, neighborList = set()):
    # Remove the occupied NN positions from neighborList.
    if (X, Y) in neighborList:
        neighborList.remove((X, Y))
    if Y > 0 and grid[X][Y-1] == 0:
        neighborList.add((X, Y-1))
    if X > 0 and grid[X-1][Y] == 0:
        neighborList.add((X-1, Y))
    if Y < len(grid[0])-1 and grid[X][Y+1] == 0:
        neighborList.add((X, Y+1))
    if X < len(grid)-1 and grid[X+1][Y] == 0:
        neighborList.add((X+1, Y))
    return neighborList

# Place Qubits

In [40]:
def place_qubits(pq, M, N, adj_mat):
    grid = [[0 for _ in range(N)] for _ in range(M)]
    placedQubitList = []

    # Place the first qubit in the center of the grid.
    grid[M//2][N//2] = pq.pop(0)
    placedQubitList.append((M//2, N//2))
    neighborList = update_neighbor_list(grid, M//2, N//2)
    # print(neighborList, placedQubitList)

    # Place the remaining qubits.
    while pq:
        q = pq.pop(0)
        minCost, minX, minY = int(1e9), int(1e9), int(1e9)
        for neighbor in neighborList:
            cost = 0
            for qubit in placedQubitList:
                cost += ManDist(neighbor, qubit) * DoA(q, qubit, grid, adj_mat)
            if cost < minCost:
                minCost = cost
                minX, minY = neighbor
            elif cost == minCost:
                if abs(neighbor[0] - M//2) + abs(neighbor[1] - N//2) < abs(minX - M//2) + abs(minY - N//2):
                    minX, minY = neighbor
            # print(f"qubit: {q}, neighbor: {neighbor}, cost: {cost}, minCost: {minCost}, minX: {minX}, minY: {minY}")
        grid[minX][minY] = q
        placedQubitList.append((minX, minY))
        neighborList = update_neighbor_list(grid, minX, minY, neighborList)
        # print(neighborList, placedQubitList)

    return grid


# Finding the best paths

In [41]:
# Check if cell(x,y) is in the grid
def is_valid_cell(grid, x, y):
    M, N = len(grid), len(grid[0])
    return 0 <= x < M and 0 <= y < N

def find_min_paths(grid, src, dest):
    M, N = len(grid), len(grid[0])
    directions = [(0, -1), (-1, 0), (0, 1), (1, 0)]
    queue = deque([(src, [src])])
    min_paths = []
    visited = set()

    while queue:
        (x, y), path = queue.popleft()
        if (x, y) == dest:
            min_paths.append(path)
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_valid_cell(grid, nx, ny) and (nx, ny) not in visited:
                queue.append(((nx, ny), path + [(nx, ny)]))
        visited.add((x, y))
    return min_paths
    

# Rearrange Utils

In [42]:
def nPaths(p1, p2):
    x1, y1 = p1[0], p1[1]
    x2, y2 = p2[0], p2[1]
    right_moves = abs(y1-y2)
    down_moves = abs(x1-x2)
    total_moves = right_moves + down_moves
    return int(math.factorial(total_moves) / (math.factorial(right_moves) * math.factorial(down_moves)))

def nTrace(p1, p2):
    x1, y1 = p1[0], p1[1]
    x2, y2 = p2[0], p2[1]
    return abs(x1-x2) + abs(y1-y2) -1

def swap_qubits(grid, path):
    n = len(path) - 1
    grids = []
    for i in range(n):
        new_grid = copy.deepcopy(grid)
        swap_src, swap_dest = i, n-i-1
        count = 0

        for j in range(swap_src):
            x1, y1 = path[j][0], path[j][1]
            x2, y2 = path[j+1][0], path[j+1][1]
            new_grid[x1][y1], new_grid[x2][y2] = new_grid[x2][y2], new_grid[x1][y1]
            count += 1
        
        for j in range(swap_dest):
            x1, y1 = path[n-j][0], path[n-j][1]
            x2, y2 = path[n-j-1][0], path[n-j-1][1]
            new_grid[x1][y1], new_grid[x2][y2] = new_grid[x2][y2], new_grid[x1][y1]
            count += 1

        grids.append(new_grid)

    return grids

# Gate Level Interaction Routing

In [43]:
def rearrange_grid(grid, q1, q2, swaps):
    p1 = find_qubit(q1, grid)
    p2 = find_qubit(q2, grid)

    np = nPaths(p1, p2)
    nt = nTrace(p1, p2)
    pathList = find_min_paths(grid, p1, p2)
    # print(f"np: {np}, nt: {nt}, pathList: {pathList}")

    all_possible_grids = []
    for path in pathList:
        new_grids = swap_qubits(copy.deepcopy(grid), path)
        for new_grid in new_grids:
            all_possible_grids.append(new_grid)
    return (grid, all_possible_grids, swaps+nt)


# Main Function

In [44]:
filename = input("Enter the filename: ")
# filename = 'cnt3-5_180'
with open(f'benchmarks/revlib_decomposed/{filename}.real', 'r') as f:
    data = f.read()

qubits = int(re.search(r".numvars ([\d]*)", data).group(1))
gates = re.search(r"(?s)(?<=.begin).*?(?=.end)", data, re.MULTILINE).group(0)
lines = gates.strip().split('\n')

# print(qubits, lines)
adj_mat = [[0]*qubits for _ in range(qubits)]
counts = {i: 0 for i in range(1, qubits+1)}
interactions = []

for t, line in enumerate(lines, 1):
    params = line.split(' ')
    if len(params) == 2: continue    

    qubit1 = int(re.search(r"\d+", params[1]).group(0))
    qubit2 = int(re.search(r"\d+", params[2]).group(0))
    
    interactions.append((qubit1, qubit2))
    adj_mat[qubit1-1][qubit2-1] += 1/t

for i in range(qubits):
    for j in range(qubits):
        val = adj_mat[i][j]
        counts[i+1] += math.ceil(val)
        counts[j+1] += math.ceil(val)

print('AdjMat:')
for row in adj_mat: print(row)
# print('Counts:', counts)
qubit_priority = []
interaction_factor = lambda x: sum(adj_mat[x-1]) + sum(row[x-1] for row in adj_mat)

possible_src = [k for k, v in counts.items() if v == max(counts.values())]
qubit_priority.append(max(possible_src, key=interaction_factor))
counts.pop(qubit_priority[0])
stack = []

while counts:
    last_qubit = qubit_priority[-1]
    temp_stack = []
    for k, v in counts.items():
        val = adj_mat[last_qubit-1][k-1] + adj_mat[k-1][last_qubit-1]
        if val != 0:
            temp_stack.append((k, val))
    
    if temp_stack:
        temp_stack.sort(key=lambda x: x[1])
        # max_qubit = temp_stack.pop(0)[0]
        for (k, v) in temp_stack:
            if k in stack: stack.remove(k)
            stack.append(k)
    else:
        stack.sort(key=lambda x: interaction_factor(x))
    # print(stack, temp_stack)
    max_qubit = stack.pop() if stack else max(counts, key=interaction_factor)
    qubit_priority.append(max_qubit)
    counts.pop(max_qubit)

print('Priority:', qubit_priority)
M, N = map(int, input("Enter the grid size: ").split())
# M, N = 3, 6
grid = place_qubits(qubit_priority, M, N, adj_mat)
print('Grid:')
for row in grid: print(row)

# rearrange_grid(grid, 1, 5, 2)
limit = int(input("Enter the limit: "))
# limit = 69

AdjMat:
[0, 0, 0, 0.002188183807439825, 0.002183406113537118, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0.002207505518763797, 0.0022026431718061676, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0.0022222222222222222, 0.002232142857142857, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0.002242152466367713, 0.0021929824561403508, 0, 0, 0, 0, 0, 0, 0]
[0.002197802197802198, 0, 0, 0, 0, 0.029433857188725213, 0.08044834146413708, 0.0022522522522522522, 0.0022123893805309734, 0, 0, 0, 0, 0, 0]
[0, 0.0022172949002217295, 0, 0, 0, 0, 1.0141223594594875, 0, 0.0022624434389140274, 0.0022271714922048997, 0, 0, 0, 0, 0]
[0.03096791492090778, 0.08585926651964387, 0.040474220393269804, 0.025604326434047142, 0.6513888888888889, 0.04299354354040055, 0, 0.013273313867516635, 0.009891366208081152, 0.013990103618263094, 0.01316574485914227, 0.01643077672669367, 0.01232820771545852, 0.018566782682399467, 0.05608953500787238]
[0, 0, 0, 0.0022471910112359553, 0, 0, 0, 0, 0, 0, 0.002288329519450801, 0.0022988505

In [45]:
leaves = [([], grid, 0)] # (parent, grids, swaps)
for i, interaction in enumerate(interactions, 1):
    print(f"Line {i}: {interaction}")
    q1, q2 = interaction
    current, excluded = [], []
    for i in leaves:
        if ManDist(find_qubit(q1, i[1]), find_qubit(q2, i[1])) > 0:
            current.append(i)
        else:
            excluded.append(i)
    new_leaves = []

    for i in current:
        parent, grid, swaps = i
        new_parent, new_grids, new_swaps = rearrange_grid(grid, q1, q2, swaps)
        new_leaves.extend([(parent+[new_parent], new_grid, new_swaps) for new_grid in new_grids if new_swaps <= limit])

    leaves = excluded + new_leaves
    tot_leaves = len(leaves)
    if tot_leaves <= 30000:
        print(f"Leaves: {len(leaves)}")
    else:
        print(f"Leaves: > 30000, pruning {tot_leaves - 30000}...")
        leaves.sort(key=lambda x: x[2])
        leaves = leaves[:30000]

Line 1: (15, 10)
Leaves: 1
Line 2: (14, 15)
Leaves: 1
Line 3: (7, 5)
Leaves: 1
Line 4: (15, 7)
Leaves: 1
Line 5: (7, 5)
Leaves: 1
Line 6: (6, 7)
Leaves: 1
Line 7: (14, 6)
Leaves: 1
Line 8: (6, 7)
Leaves: 1
Line 9: (13, 6)
Leaves: 1
Line 10: (12, 6)
Leaves: 1
Line 11: (12, 13)
Leaves: 4
Line 12: (13, 6)
Leaves: 10
Line 13: (6, 7)
Leaves: 29
Line 14: (14, 6)
Leaves: 102
Line 15: (6, 7)
Leaves: 231
Line 16: (7, 5)
Leaves: 909
Line 17: (15, 7)
Leaves: 5225
Line 18: (7, 5)
Leaves: 14863
Line 19: (6, 7)
Leaves: > 30000, pruning 30515...
Line 20: (14, 6)
Leaves: > 30000, pruning 82740...
Line 21: (6, 7)
Leaves: > 30000, pruning 29025...
Line 22: (13, 6)
Leaves: > 30000, pruning 130166...
Line 23: (12, 13)
Leaves: > 30000, pruning 119202...
Line 24: (12, 6)
Leaves: > 30000, pruning 83810...
Line 25: (13, 6)
Leaves: > 30000, pruning 54012...
Line 26: (6, 7)
Leaves: > 30000, pruning 76393...
Line 27: (14, 6)
Leaves: > 30000, pruning 86900...
Line 28: (6, 7)
Leaves: > 30000, pruning 29057...
Line

In [46]:
# Find the minimum swaps from the current grids
min_swaps = int(1e9)
for i in leaves:
    # print(i)
    parent, grids, swaps = i
    if swaps < min_swaps:
        min_swaps = swaps
        min_grid = grids
        min_parent = parent

print(f"Minimum swaps: {min_swaps}")
print("Path: ")
for parent in min_parent:
    for row in parent: print(row)
    print()
for row in min_grid: print(row)

Minimum swaps: 132
Path: 
[11, 4, 3]
[10, 9, 8]
[15, 7, 5]
[14, 6, 12]
[1, 13, 2]

[11, 4, 3]
[10, 9, 8]
[15, 7, 5]
[14, 6, 2]
[1, 13, 12]

[11, 4, 3]
[10, 9, 8]
[15, 7, 5]
[14, 6, 12]
[1, 13, 2]

[11, 4, 3]
[10, 9, 8]
[14, 7, 5]
[6, 15, 12]
[1, 13, 2]

[11, 4, 3]
[10, 9, 8]
[7, 14, 5]
[6, 15, 12]
[1, 13, 2]

[11, 4, 3]
[10, 9, 8]
[7, 15, 5]
[6, 14, 12]
[1, 13, 2]

[11, 4, 3]
[10, 9, 8]
[7, 15, 12]
[6, 14, 5]
[1, 13, 2]

[11, 4, 3]
[10, 9, 8]
[15, 7, 2]
[6, 14, 12]
[1, 13, 5]

[11, 4, 3]
[10, 9, 8]
[15, 7, 2]
[14, 6, 12]
[1, 13, 5]

[11, 4, 3]
[10, 9, 8]
[15, 7, 2]
[14, 6, 5]
[1, 13, 12]

[11, 4, 3]
[10, 9, 8]
[15, 7, 2]
[14, 6, 12]
[1, 13, 5]

[11, 4, 3]
[15, 9, 8]
[10, 7, 2]
[14, 6, 12]
[1, 13, 5]

[11, 4, 3]
[9, 15, 8]
[10, 7, 2]
[14, 6, 12]
[1, 13, 5]

[11, 4, 3]
[9, 15, 8]
[10, 6, 2]
[14, 7, 12]
[1, 13, 5]

[11, 4, 3]
[9, 6, 8]
[10, 15, 2]
[14, 7, 12]
[1, 13, 5]

[11, 4, 3]
[9, 6, 8]
[10, 15, 2]
[14, 7, 5]
[1, 13, 12]

[11, 4, 3]
[9, 6, 8]
[10, 15, 2]
[14, 7, 12]
[1, 13, 5]

[11, 